In [2]:
# Install needed stuff to the Jupyter and kernel environment
# Need to install only once, so commented out for cleaner output
# %pip install pyarrow
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install pyfinance

In [3]:
# Import needed stuff
import sys
import pyarrow.feather as feather
import math
import pandas
import numpy as np
import matplotlib.pyplot as plt
from pyfinance.options import BSM
from datetime import datetime, timedelta, date

# Hedging assigment

### Data fetching and wrangling

In [5]:
apple_data_df = pandas.read_feather('apple.feather')

         Date  Underlying   C115   C120   C125   C130   C135  P115  P120  \
0  2021-12-20      169.75   <NA>  51.05   45.6   41.4  36.68   0.7  0.88   
1  2021-12-21      172.99   55.4   51.7  48.25     42  38.01  0.53  0.66   
2  2021-12-22      175.64   59.9   56.0   51.0   45.9  41.95  0.52   0.6   
3  2021-12-23      176.28   <NA>   57.1   <NA>   47.1  42.58  0.43  0.53   
4  2021-12-27      180.33  65.56   60.2  55.75   51.1  45.07  0.36  0.45   
..        ...         ...    ...    ...    ...    ...    ...   ...   ...   
57 2022-03-14      150.62   <NA>   32.4  25.35  20.79   15.8  0.03  0.04   
58 2022-03-15      155.09  37.95   35.1  29.03   25.2  20.14  0.02  0.02   
59 2022-03-16      159.59  39.75  37.05  32.58   29.2   24.3  0.01  0.01   
60 2022-03-17      160.62   45.0   40.2  35.65  30.22  25.75  0.01  0.01   
61 2022-03-18      163.98  47.27  44.08   39.0   33.5  28.58  <NA>  0.01   

    P125  P130  P135  
0   1.08   1.4  1.78  
1   0.83  1.08  1.34  
2    0.7  0.91  1.

## The hedging function itself

In [9]:
def delta_hedge(data):
    print(data)
    return
    # Get time to maturity in years
    t = 46/365 # Comes from the option expiration date counted backwards
    # Debug print
    print(t)

    # t = 0 is 2.10.
    t0 = '2023-10-02'

    # US treasury 3-Month Treasury Yield, Level from FRED for Q2/2023
    r = 0.05076
    
    initial_call = BSM(kind='call', S0=underlying_data.iloc[0]['Close'], K=call_option_data.iloc[0]['strike'], T=t, r=r, sigma=call_option_data.iloc[0]['impliedVolatility'])
    print('Initial call option value from BSM', initial_call.value()) # Value of the BSM for the call option c0 at time t0

    initial_delta = initial_call.delta() # Delta of the BSM for the call option c0 at time t0
    print('Initial value of delta from BSM', initial_delta)

    scraped_data = call_option_data[['strike', 'impliedVolatility']]
    scraped_strike_prices_close_to_atm = scraped_data.iloc[100:110]
    print(scraped_strike_prices_close_to_atm)
    for entry in underlying_data['Close']:
        for index, row in scraped_strike_prices_close_to_atm.iterrows():
            call = BSM(kind='call', S0=entry, K=row["strike"], T=t, r=r, sigma=row['impliedVolatility'])
            print('Call option value from BSM', call.value()) # Value of the BSM for the call option c at time t
        
            delta = call.delta() # Delta of the BSM for the call option c at time t
            print('Value of delta from BSM', delta)
            print('\n')
            

In [10]:
delta_hedge(apple_data_df)

         Date  Underlying   C115   C120   C125   C130   C135  P115  P120  \
0  2021-12-20      169.75   <NA>  51.05   45.6   41.4  36.68   0.7  0.88   
1  2021-12-21      172.99   55.4   51.7  48.25     42  38.01  0.53  0.66   
2  2021-12-22      175.64   59.9   56.0   51.0   45.9  41.95  0.52   0.6   
3  2021-12-23      176.28   <NA>   57.1   <NA>   47.1  42.58  0.43  0.53   
4  2021-12-27      180.33  65.56   60.2  55.75   51.1  45.07  0.36  0.45   
..        ...         ...    ...    ...    ...    ...    ...   ...   ...   
57 2022-03-14      150.62   <NA>   32.4  25.35  20.79   15.8  0.03  0.04   
58 2022-03-15      155.09  37.95   35.1  29.03   25.2  20.14  0.02  0.02   
59 2022-03-16      159.59  39.75  37.05  32.58   29.2   24.3  0.01  0.01   
60 2022-03-17      160.62   45.0   40.2  35.65  30.22  25.75  0.01  0.01   
61 2022-03-18      163.98  47.27  44.08   39.0   33.5  28.58  <NA>  0.01   

    P125  P130  P135  
0   1.08   1.4  1.78  
1   0.83  1.08  1.34  
2    0.7  0.91  1.